<a href="https://colab.research.google.com/github/MonikaRudra/FDPLLM/blob/main/Monika_Next_word_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
text="this is a python class and we all are here to learn this new skill. In this session we are going"

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [ ]:
class customDataset(Dataset):
     def __init__(self,text,word2idx,seq_length):
      self.text = text
      self.word2idx = word2idx
      self.seq_length = seq_length

     def __len__(self):
      return len(self.text)

     def __getitem__(self,index):
      sequence = [self.word2idx[word] for word in self.text[index:index+self.seq_length]]
      target = self.word2idx[self.text[index+self.seq_length]]

      return torch.tensor(sequence), torch.tensor(target)

In [ ]:
word2idx = { word:i for i, word in enumerate(set(text.split()))}

In [ ]:
word2idx

{'In': 0,
 'skill.': 1,
 'are': 2,
 'here': 3,
 'learn': 4,
 'python': 5,
 'going': 6,
 'all': 7,
 'new': 8,
 'this': 9,
 'class': 10,
 'is': 11,
 'to': 12,
 'session': 13,
 'a': 14,
 'we': 15,
 'and': 16}

In [ ]:
idx2word = {i: word for word, i in word2idx.items()}

In [ ]:
idx2word

{0: 'In',
 1: 'skill.',
 2: 'are',
 3: 'here',
 4: 'learn',
 5: 'python',
 6: 'going',
 7: 'all',
 8: 'new',
 9: 'this',
 10: 'class',
 11: 'is',
 12: 'to',
 13: 'session',
 14: 'a',
 15: 'we',
 16: 'and'}

In [ ]:
dataset = customDataset(text.split(),word2idx,seq_length=10)

In [ ]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
class VanillaRNN(nn.Module):
  def __init__(self, vocab_size, embed_size, hidden_size):
    super(VanillaRNN, self).__init__()
    self.embed = nn.Embedding(vocab_size, embed_size)
    self.rnn = nn.RNN(embed_size,hidden_size,batch_first=True)
    self.fc = nn.Linear(hidden_size,vocab_size)

  def forward(self, x, h0):
    embed = self.embed(x)
    out,h = self.rnn(embed, h0)
    output = self.fc(out[:,-1,:])
    return output, h

In [ ]:
model = VanillaRNN(len(word2idx),embed_size=128,hidden_size=256 )

In [ ]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr =0.001)

In [ ]:
for epoch in range(10):
  for input,label in dataloader:
    optimizer.zero_grad()
    h0 = torch.zeros(1,input.size(0),256)
    outputs, _ = model(input,h0)
    loss = criterion(outputs,label)
    loss.backward()
    optimizer.step()
  print(f"Epoch {epoch} : Loss : {loss.item()}")

input_seq = torch.tensor([word2idx[word] for word in text.split()[-10:]]).unsqueeze(0)
h0 = torch.zeros(1,1,256)
ouput, _ = model(input_seq,h0)
predicted_word = idx2word[ouput.argmax().item()]
print(f"Predicted next word: {predicted_word}")

AttributeError: 'int' object has no attribute 'index'